# Generation of a cosine similarity matrix to compute movie similarity based on movie features

To identify the diversity for a list of recommendations, it is crucial to first determine the intra-list similarity (IL) between the items, since diversity is defined as the inverse of the IL.


diversity = 1 - IL


The IL can be calculated by averaging the similarity of each movie to another on the list.

To shorten the process of calculating the IL, we generate a cosine similarity matrix for each movie to another in the MovieLens dataset. The cosine similarity matrix will be further used when generating actual recommendations for users. It contains cosine similarities computed in the content-based recommender with similarities based the text of the description, keywords and genre. The used movie features derive from *The Movies Database* (tmdb).

## Import of the libaries

In [99]:
import pandas as pd
import numpy as np


import scipy.sparse as sp
from typing import List

from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer, PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Construction of the feature dataframe for the similarity measurement

## Load data from tmdb dataset

In [100]:
df_meta = pd.read_csv("../content/movies_metadata.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [101]:
df_meta.head()


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [102]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [103]:
df_meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [104]:
# load credit and keyword table
df_credits = pd.read_csv("../content/credits.csv", engine="python")
df_keywords = pd.read_csv("../content/keywords.csv")

In [105]:
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [106]:
df_keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [107]:
df_keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


## Cleaning data (tmdb credit and keyword df)

In [108]:
# Merging keywords and credits df
df_features = df_keywords.merge(df_credits, on="id")
df_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46496 entries, 0 to 46495
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46496 non-null  int64 
 1   keywords  46496 non-null  object
 2   cast      46496 non-null  object
 3   crew      46496 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.8+ MB


In [109]:
#apply literal_eval from Abstract Syntax Trees (ast) to convert str into a list of dictionaries
df_features['cast'] = df_features['cast'].apply(literal_eval)
df_features['crew'] = df_features['crew'].apply(literal_eval)
df_features['keywords'] = df_features['keywords'].apply(literal_eval)

In [110]:
#create function to get list of lists
def get_lists(key, column, df):
    x = []

    for i in range(df.shape[0]):
        values = [d[f"{key}"] for d in df[f"{column}"][i]]
        x.append(values)
    return x

In [111]:
#add new columns to features data frame
df_features["crew_jobs"] = pd.Series(get_lists("job", "crew", df_features))
df_features["actors"] = pd.Series(get_lists("name", "cast", df_features))
df_features["keywords"] = pd.Series(get_lists("name", "keywords", df_features))

In [112]:
# select only cleaned columns for the new dataset
df_features = df_features[["id", "actors", "keywords", "crew_jobs"]]
df_features.head()

,id,actors,keywords,crew_jobs
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[jealousy, toy, boy, friendship, friends, riva...","[Director, Screenplay, Screenplay, Screenplay,..."
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[board game, disappearance, based on children'...","[Executive Producer, Screenplay, Original Musi..."
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[fishing, best friend, duringcreditsstinger, o...","[Director, Characters, Writer, Sound Recordist]"
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...","[based on novel, interracial relationship, sin...","[Director, Screenplay, Producer, Producer, Pro..."
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[baby, midlife crisis, confidence, aging, daug...","[Original Music Composer, Director of Photogra..."


## Cleaning data from tmdb metadata df

In [113]:
df_meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## Selecting features from the dataframe, that are suitable for computing content-based similarity

For the calculation of the cosine similarity are following features considered:
+ title
+ overview (description of the movie), 
+ genre
+ actors
+ release year
+ director
+ writer
+ keywords


In [114]:
# CHeckout all features, that are available in the meta dataframe
df_meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [115]:
# make a new datafame with only desired features
meta = df_meta[['genres','id', 'overview', 'release_date', 'tagline', 'title']]
meta.head()

,genres,id,overview,release_date,tagline,title
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,NaN,Toy Story
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,When siblings Judy and Peter discover an encha...,1995-12-15,Roll the dice and unleash the excitement!,Jumanji
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,A family wedding reignites the ancient feud be...,1995-12-22,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Friends are the people who let you be yourself...,Waiting to Exhale
4,"[{'id': 35, 'name': 'Comedy'}]",11862,Just when George Banks has recovered from his ...,1995-02-10,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II


In [116]:
#apply literal_eval from Abstract Syntax Trees (ast) to convert str into a list of dictionaries
meta['genres'] = meta['genres'].apply(literal_eval)

# add new columns to features data frame
meta['genres'] = pd.Series(get_lists("name", "genres", meta))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


### Extract the release year

In [117]:
# convert release date to datetime object
meta["release_date"] = pd.to_datetime(meta["release_date"], errors="coerce")

# extract the year
meta["release_date"] = (pd.DatetimeIndex(meta["release_date"]).year)

# cast the year from float into integer
meta["release_date"] = meta["release_date"].astype("Int64")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [118]:
# check the dataframe
meta.head()

,genres,id,overview,release_date,tagline,title
0,"[Animation, Comedy, Family]",862,"Led by Woody, Andy's toys live happily in his ...",1995,NaN,Toy Story
1,"[Adventure, Fantasy, Family]",8844,When siblings Judy and Peter discover an encha...,1995,Roll the dice and unleash the excitement!,Jumanji
2,"[Romance, Comedy]",15602,A family wedding reignites the ancient feud be...,1995,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men
3,"[Comedy, Drama, Romance]",31357,"Cheated on, mistreated and stepped on, the wom...",1995,Friends are the people who let you be yourself...,Waiting to Exhale
4,[Comedy],11862,Just when George Banks has recovered from his ...,1995,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II


## Merge the metadata with the df_feature

In [119]:
# drop rows with incorrect tmdb id´s 
meta = meta.drop([19730, 29503, 35587])

# cast id in meta to int
meta["id"] = meta["id"].astype(int)

In [120]:
# Merging meta and df_feature table on tmdb id
df_features = df_features.merge(meta, on="id")
df_features.head()

,id,actors,keywords,crew_jobs,genres,overview,release_date,tagline,title
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[jealousy, toy, boy, friendship, friends, riva...","[Director, Screenplay, Screenplay, Screenplay,...","[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...",1995,NaN,Toy Story
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[board game, disappearance, based on children'...","[Executive Producer, Screenplay, Original Musi...","[Adventure, Fantasy, Family]",When siblings Judy and Peter discover an encha...,1995,Roll the dice and unleash the excitement!,Jumanji
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[fishing, best friend, duringcreditsstinger, o...","[Director, Characters, Writer, Sound Recordist]","[Romance, Comedy]",A family wedding reignites the ancient feud be...,1995,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...","[based on novel, interracial relationship, sin...","[Director, Screenplay, Producer, Producer, Pro...","[Comedy, Drama, Romance]","Cheated on, mistreated and stepped on, the wom...",1995,Friends are the people who let you be yourself...,Waiting to Exhale
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[baby, midlife crisis, confidence, aging, daug...","[Original Music Composer, Director of Photogra...",[Comedy],Just when George Banks has recovered from his ...,1995,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II


## Set movielens ID as key for the feature dataframe
Since the recommendation lists from the collaborative filtering derives from the movielens dataset, the df_features has to have the movielens Id as index for the movies. Further, the tmdb dataset contains much more movies than the movielens dataset. By an inner merge only the intersection from both frames is preserved and the desired movies from the movieslens dataset remain in the table.

In [121]:
# Load the links table from MovieLens
df_links = pd.read_csv("../content/links.csv")

In [122]:
# Checkout the imported dataframe
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [123]:
df_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [124]:
# To avoid further problems with the datatyp of the movieId, it is casted into a string
df_links.movieId.astype(str)

0            1
1            2
2            3
3            4
4            5
         ...  
9737    193581
9738    193583
9739    193585
9740    193587
9741    193609
Name: movieId, Length: 9742, dtype: object

In [125]:
#check for duplicates
df_links.movieId.nunique()

9742

In [126]:
#check for duplicates
df_links.tmdbId.nunique()

9733

There are 8 duplicates in movieId and 9 Duplicates in the tmdbId. The imdbId is not of interest here and will not be used further.

In [127]:
# remove the duplicates
df_links.drop_duplicates(subset="movieId", inplace=True)
df_links.drop_duplicates(subset="tmdbId", inplace=True)
df_links.shape

(9734, 3)

In [128]:
# check for null values
df_links["tmdbId"].isnull().sum()

1

In [129]:
df_links.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9734 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9734 non-null   int64  
 1   imdbId   9734 non-null   int64  
 2   tmdbId   9733 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 304.2 KB


In [130]:
# drop the null values
df_links.dropna(inplace=True)

In [131]:
# rename the id in the df_features to tmdbId for merging
df_features.rename(columns={"id":"tmdbId"}, inplace=True)

In [132]:
# cast the tmdbId into integer for merging
df_links["tmdbId"] = df_links["tmdbId"].astype("Int64")
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862


### Merging the feature dataframe with MovieLens links table

In [133]:
# merge the feature dataframe with the link table from movielens 
df_features = df_features.merge(df_links,on="tmdbId")
df_features.head()

,tmdbId,actors,keywords,crew_jobs,genres,overview,release_date,tagline,title,movieId,imdbId
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[jealousy, toy, boy, friendship, friends, riva...","[Director, Screenplay, Screenplay, Screenplay,...","[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...",1995,NaN,Toy Story,1,114709
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[board game, disappearance, based on children'...","[Executive Producer, Screenplay, Original Musi...","[Adventure, Fantasy, Family]",When siblings Judy and Peter discover an encha...,1995,Roll the dice and unleash the excitement!,Jumanji,2,113497
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[fishing, best friend, duringcreditsstinger, o...","[Director, Characters, Writer, Sound Recordist]","[Romance, Comedy]",A family wedding reignites the ancient feud be...,1995,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,3,113228
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...","[based on novel, interracial relationship, sin...","[Director, Screenplay, Producer, Producer, Pro...","[Comedy, Drama, Romance]","Cheated on, mistreated and stepped on, the wom...",1995,Friends are the people who let you be yourself...,Waiting to Exhale,4,114885
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[baby, midlife crisis, confidence, aging, daug...","[Original Music Composer, Director of Photogra...",[Comedy],Just when George Banks has recovered from his ...,1995,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5,113041


In [134]:
# check for duplicates in movieId
df_features.movieId.duplicated().sum()

114

In [135]:
# remove duplicates
df_features.drop_duplicates(subset="movieId", inplace=True)

In [136]:
# set the movieId as index
df_features = df_features.set_index("movieId")
df_features.head()

,tmdbId,actors,keywords,crew_jobs,genres,overview,release_date,tagline,title,imdbId
movieId,,,,,,,,,,
1,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[jealousy, toy, boy, friendship, friends, riva...","[Director, Screenplay, Screenplay, Screenplay,...","[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...",1995,NaN,Toy Story,114709
2,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[board game, disappearance, based on children'...","[Executive Producer, Screenplay, Original Musi...","[Adventure, Fantasy, Family]",When siblings Judy and Peter discover an encha...,1995,Roll the dice and unleash the excitement!,Jumanji,113497
3,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[fishing, best friend, duringcreditsstinger, o...","[Director, Characters, Writer, Sound Recordist]","[Romance, Comedy]",A family wedding reignites the ancient feud be...,1995,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,113228
4,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...","[based on novel, interracial relationship, sin...","[Director, Screenplay, Producer, Producer, Pro...","[Comedy, Drama, Romance]","Cheated on, mistreated and stepped on, the wom...",1995,Friends are the people who let you be yourself...,Waiting to Exhale,114885
5,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[baby, midlife crisis, confidence, aging, daug...","[Original Music Composer, Director of Photogra...",[Comedy],Just when George Banks has recovered from his ...,1995,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,113041


In [137]:
df_features.tail()

,tmdbId,actors,keywords,crew_jobs,genres,overview,release_date,tagline,title,imdbId
movieId,,,,,,,,,,
175705,7014,"[Michel Piccoli, Miou-Miou, Béatrice Romand, F...",[absurdism],"[Sound Designer, Director, Novel, Producer, Pr...",[Comedy],"Made without proper language, just gibberish a...",1973,NaN,Themroc,69369
175707,411516,[Brunhilde Pomsel],[],"[Director, Director, Director, Director, Write...",[Documentary],"Brunhilde Pomsel describes herself as an ""apol...",2016,NaN,A German Life,5135434
175743,433410,"[Julian Radlmaier, Deragh Campbell, Beniamin F...",[],"[Director, Writer]",[Comedy],A bourgeois dog confesses how he was transform...,2017,NaN,Self-criticism of a Bourgeois Dog,6354108
175781,28469,[Helmut Qualtinger],[],[],[Comedy],No overview found.,1961,NaN,Der Herr Karl,273646
176051,460135,"[Grey Griffin, Tara Strong, Anais Fairweather,...","[superhero, lego]","[Producer, Executive Producer, Executive Produ...",[Animation],"When Supergirl, Wonder Woman, Batgirl, Bumbleb...",2017,NaN,LEGO DC Super Hero Girls: Brain Drain,7158814


In [138]:
df_features.shape

(9543, 10)

## Fill missing values with empty space, so no error occur in the NLP

In [139]:
# check out missing values
df_features.isnull().sum()

tmdbId             0
actors             0
keywords           0
crew_jobs          0
genres             0
overview          18
release_date       2
tagline         2303
title              0
imdbId             0
dtype: int64

In [140]:
# replace missing values in text with space
df_features["tagline"] = df_features["tagline"].fillna(" ")
df_features["overview"] = df_features["overview"].fillna(" ")

In [141]:
# look for index of the missing values in release_date
rows_with_nan = [index for index, row in df_features.iterrows() if row.isnull().any()]
rows_with_nan

[86237, 171495]

In [142]:
# check out the rows with missing values
df_features.loc[86237]

tmdbId                                                     367647
actors                                                         []
keywords                                             [miniseries]
crew_jobs                                                      []
genres                                                         []
overview        Documentary  Follow James Burke through the hi...
release_date                                                 <NA>
tagline                                                          
title                                                 Connections
imdbId                                                      78588
Name: 86237, dtype: object

In [143]:
# check out the rows with missing values
df_features.loc[171495]

tmdbId                                                     409926
actors                                                         []
keywords                                                       []
crew_jobs                                                      []
genres                                                         []
overview        Astronomer Dr. Carl Sagan is host and narrator...
release_date                                                 <NA>
tagline                                                          
title                                                      Cosmos
imdbId                                                      81846
Name: 171495, dtype: object

## Stemming of the words in the description

In [144]:
# split the overview text into single words
df_features["overview"] = df_features["overview"].str.split()

In [145]:
df_features.head()

,tmdbId,actors,keywords,crew_jobs,genres,overview,release_date,tagline,title,imdbId
movieId,,,,,,,,,,
1,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[jealousy, toy, boy, friendship, friends, riva...","[Director, Screenplay, Screenplay, Screenplay,...","[Animation, Comedy, Family]","[Led, by, Woody,, Andy's, toys, live, happily,...",1995,,Toy Story,114709
2,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[board game, disappearance, based on children'...","[Executive Producer, Screenplay, Original Musi...","[Adventure, Fantasy, Family]","[When, siblings, Judy, and, Peter, discover, a...",1995,Roll the dice and unleash the excitement!,Jumanji,113497
3,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[fishing, best friend, duringcreditsstinger, o...","[Director, Characters, Writer, Sound Recordist]","[Romance, Comedy]","[A, family, wedding, reignites, the, ancient, ...",1995,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,113228
4,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...","[based on novel, interracial relationship, sin...","[Director, Screenplay, Producer, Producer, Pro...","[Comedy, Drama, Romance]","[Cheated, on,, mistreated, and, stepped, on,, ...",1995,Friends are the people who let you be yourself...,Waiting to Exhale,114885
5,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[baby, midlife crisis, confidence, aging, daug...","[Original Music Composer, Director of Photogra...",[Comedy],"[Just, when, George, Banks, has, recovered, fr...",1995,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,113041


In [146]:
# Initializing stemmer and countvectorizer 
ps_stemmer = SnowballStemmer('english')#PorterStemmer()
#apply stemmer to pd.Series
df_features['stemmed_overview'] = df_features['overview'].apply(lambda x: [ps_stemmer.stem(y) for y in x])


In [147]:
# join the list of actors, keywords and genre
df_features["actors"] = df_features["actors"].str.join(" ")
df_features["keywords"] = df_features["keywords"].str.join(" ")
df_features["genres"] = df_features["genres"].str.join(" ")
df_features["stemmed_overview"] = df_features["stemmed_overview"].str.join(" ")
df_features.head()

,tmdbId,actors,keywords,crew_jobs,genres,overview,release_date,tagline,title,imdbId,stemmed_overview
movieId,,,,,,,,,,,
1,862,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,jealousy toy boy friendship friends rivalry bo...,"[Director, Screenplay, Screenplay, Screenplay,...",Animation Comedy Family,"[Led, by, Woody,, Andy's, toys, live, happily,...",1995,,Toy Story,114709,"led by woody, andi toy live happili in his roo..."
2,8844,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,board game disappearance based on children's b...,"[Executive Producer, Screenplay, Original Musi...",Adventure Fantasy Family,"[When, siblings, Judy, and, Peter, discover, a...",1995,Roll the dice and unleash the excitement!,Jumanji,113497,when sibl judi and peter discov an enchant boa...
3,15602,Walter Matthau Jack Lemmon Ann-Margret Sophia ...,fishing best friend duringcreditsstinger old men,"[Director, Characters, Writer, Sound Recordist]",Romance Comedy,"[A, family, wedding, reignites, the, ancient, ...",1995,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,113228,a famili wed reignit the ancient feud between ...
4,31357,Whitney Houston Angela Bassett Loretta Devine ...,based on novel interracial relationship single...,"[Director, Screenplay, Producer, Producer, Pro...",Comedy Drama Romance,"[Cheated, on,, mistreated, and, stepped, on,, ...",1995,Friends are the people who let you be yourself...,Waiting to Exhale,114885,"cheat on, mistreat and step on, the women are ..."
5,11862,Steve Martin Diane Keaton Martin Short Kimberl...,baby midlife crisis confidence aging daughter ...,"[Original Music Composer, Director of Photogra...",Comedy,"[Just, when, George, Banks, has, recovered, fr...",1995,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,113041,just when georg bank has recov from his daught...


In [148]:
df_features[df_features.title.duplicated()]

,tmdbId,actors,keywords,crew_jobs,genres,overview,release_date,tagline,title,imdbId,stemmed_overview
movieId,,,,,,,,,,,
915,6620,Humphrey Bogart Audrey Hepburn William Holden ...,brother brother relationship chauffeur champag...,"[Director, Screenplay, Screenplay, Producer, D...",Comedy Drama Romance,"[Linus, and, David, Larrabee, are, the, two, s...",1954,...the chauffeur's daughter who learned her st...,Sabrina,47437,linus and david larrabe are the two son of a v...
1344,11349,Gregory Peck Robert Mitchum Polly Bergen Lori ...,poison small town boat ex-detainee psychopath ...,"[Director, Novel, Screenplay, Producer, Origin...",Drama Thriller,"[Sam, Bowden, witnesses, a, rape, committed, b...",1962,Now he had only one weapon left - murder! ...T...,Cape Fear,55824,sam bowden wit a rape commit by max cadi and t...
1726,9922,Kevin Costner Will Patton Olivia Williams Lare...,usa post postman army apocalypse,"[Screenplay, Producer, Original Music Composer...",Drama Adventure,"[In, 2013, there, are, no, highways,, no, I-wa...",1997,The year is 2013. One man walked in off the ho...,The Postman,119925,"in 2013 there are no highways, no i-ways, no d..."
1941,23383,Laurence Olivier Jean Simmons John Laurie Esmo...,shakespeare father murder,"[Director, Screenplay, Novel, Executive Producer]",Drama,"[Winner, of, four, Academy, Awards,, including...",1948,,Hamlet,40416,"winner of four academi awards, includ best pic..."
2059,9820,Lindsay Lohan Dennis Quaid Natasha Richardson ...,matchmaking divorced twins separated at birth ...,"[Original Music Composer, Production Design, D...",Comedy Drama Family,"[Hallie, Parker, and, Annie, James, are, ident...",1998,"Twice the Fun, Double the Trouble.",The Parent Trap,120783,halli parker and anni jame are ident twin sepa...
...,...,...,...,...,...,...,...,...,...,...,...
169982,305470,Dacre Montgomery Naomi Scott RJ Cyler Becky G ...,spaceship based on tv series tokusatsu superhe...,"[Producer, Producer, Producer, Producer, Produ...",Action Adventure Science Fiction,"[Saban's, Power, Rangers, follows, five, ordin...",2017,Together we are more,Power Rangers,3717490,saban power ranger follow five ordinari teen w...
170827,282035,Tom Cruise Russell Crowe Annabelle Wallis Sofi...,monster mummy horror,"[Casting, Executive Producer, Executive Produc...",Thriller Action Adventure,"[Though, safely, entombed, in, a, crypt, deep,...",2017,Welcome To A New World of Gods And Monsters,The Mummy,2345759,though safe entomb in a crypt deep beneath the...
172253,38966,Keanu Reeves Lori Loughlin Theresa Saldana Tri...,handcuffs amnesia tied up,"[Director, Writer, Camera Operator]",Comedy,"[The, Night, Before, is, a, 1988, film, starri...",1988,You lost your father's car. Sold your prom dat...,The Night Before,95730,the night befor is a 1988 film star keanu reev...


In [149]:
df_features.index.nunique()

9543

## Export the feature dataframe
unhash the code below, if you want to export the feature table and change the directory path accordingly

In [150]:
# export new data to csv. file
#df_features.to_csv('../data/df_features.csv')

## Make a combined feature of texts


In [151]:
#def combined_features(row):
df_features["combined_features"] = df_features.stemmed_overview + df_features.genres + df_features.keywords

In [152]:
# check out an example of a stemmed overview (Toy Story)
df_features.combined_features[1]

'led by woody, andi toy live happili in his room until andi birthday bring buzz lightyear onto the scene. afraid of lose his place in andi heart, woodi plot against buzz. but when circumst separ buzz and woodi from their owner, the duo eventu learn to put asid their differences.Animation Comedy Familyjealousy toy boy friendship friends rivalry boy next door new toy toy comes to life'

## Generate a matrix of cosine similarity of the features

In [153]:
#use CountVectorizer for NLP
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')

count_matrix = count.fit_transform(df_features['combined_features'].values.astype('U'))

#calculate similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)

### Make sample recommendations
In order to test, if the cosine similarity computation result in meaningful recommendations, a few sample recommendations are made.

In [154]:
# Function to get TOP10 recommendations according to a seed item
def get_cb_recommendations(title, df_features, cosine_sim):
    ''' compares cosine similarity between movies and ranks the movies according to the score, 
    returns a list of movie titles for recommendation

    '''
    df_features = df_features.reset_index()
    titles = df_features['title']
    indices = pd.Series(df_features.index, index=df_features['title'])
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [155]:
# get a sample recommendation to check the result, seed item "Toy Story"
get_cb_recommendations("Toy Story", df_features, cosine_sim)

2351                   Toy Story 2
7332                   Toy Story 3
1398                Small Soldiers
1463                  Child's Play
53      The Indian in the Cupboard
5955        The 40 Year Old Virgin
1671                          Toys
1465                Child's Play 3
1464                Child's Play 2
7892                           Ted
Name: title, dtype: object

In [156]:
get_cb_recommendations("Batman Begins", df_features, cosine_sim)

6689                            The Dark Knight
509                                      Batman
1171                             Batman & Robin
7741                      The Dark Knight Rises
1058                             Batman Returns
7357                 Batman: Under the Red Hood
8004    Batman: The Dark Knight Returns, Part 1
4945                           Enter the Dragon
8792                           Batman vs. Robin
8926         Batman v Superman: Dawn of Justice
Name: title, dtype: object

In [157]:
get_cb_recommendations("Interstellar", df_features, cosine_sim)

4095                         Solaris
1902              Planet of the Apes
705            2001: A Space Odyssey
9391                      Passengers
2503                 Mission to Mars
9387    Rogue One: A Star Wars Story
3840                  Silent Running
8216                      About Time
3407              Planet of the Apes
2708                       Moonraker
Name: title, dtype: object

In [158]:
# To check the correctness of the indexing, this function retrieves the movie informations from the feature table
def get_info(title, df_features):
    '''retrieve informations about the movie from the feature table'''
    indices = pd.Series(df_features.index, index=df_features['title'])
    idx = indices[title]
    film_info = df_features.loc[idx]
    return film_info

In [159]:
get_info("Toy Story", df_features)

tmdbId                                                             862
actors               Tom Hanks Tim Allen Don Rickles Jim Varney Wal...
keywords             jealousy toy boy friendship friends rivalry bo...
crew_jobs            [Director, Screenplay, Screenplay, Screenplay,...
genres                                         Animation Comedy Family
overview             [Led, by, Woody,, Andy's, toys, live, happily,...
release_date                                                      1995
tagline                                                               
title                                                        Toy Story
imdbId                                                          114709
stemmed_overview     led by woody, andi toy live happili in his roo...
combined_features    led by woody, andi toy live happili in his roo...
Name: 1, dtype: object

### Conclusion
The movies on each recommendation list are close to the seed items. They overlap by genre, content etc. and it was even possible to identify the sequels of the seed movie.

## Save the cosine similarity array as an dataframe

In [160]:
# save the matrix in a dataframe
df_cosine_sim = pd.DataFrame(cosine_sim)
df_cosine_sim.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9503,9504,9505,9506,9507,9508,9509,9510,9511,9512,9513,9514,9515,9516,9517,9518,9519,9520,9521,9522,9523,9524,9525,9526,9527,9528,9529,9530,9531,9532,9533,9534,9535,9536,9537,9538,9539,9540,9541,9542
0,1.000000,0.033900,0.010186,0.010879,0.000000,0.000000,0.012142,0.046066,0.000000,0.000000,0.021758,0.014316,0.023869,0.000000,0.000000,0.016464,0.011036,0.044143,0.010447,0.039312,0.022530,0.000000,0.015020,0.024598,0.018065,0.015958,0.019656,0.000000,0.012836,0.025673,0.008946,0.000000,0.030880,0.000000,0.022740,0.017247,0.000000,0.000000,0.000000,0.024126,...,0.032929,0.000000,0.011737,0.000000,0.051068,0.000000,0.028633,0.000000,0.000000,0.000000,0.000000,0.023869,0.015070,0.024213,0.000000,0.014679,0.000000,0.000000,0.000000,0.000000,0.000000,0.006930,0.028215,0.000000,0.030557,0.023669,0.022233,0.09167,0.110558,0.011934,0.015958,0.000000,0.016464,0.000000,0.018334,0.014679,0.008266,0.025425,0.052926,0.000000
1,0.033900,1.000000,0.020544,0.000000,0.010148,0.030094,0.000000,0.009292,0.100504,0.009387,0.021944,0.014438,0.000000,0.010536,0.000000,0.000000,0.011130,0.044519,0.000000,0.029735,0.007574,0.000000,0.022721,0.016538,0.009109,0.016093,0.009912,0.008775,0.051782,0.000000,0.009022,0.021995,0.015571,0.000000,0.011467,0.008697,0.000000,0.000000,0.009800,0.012166,...,0.016605,0.019964,0.000000,0.014804,0.017168,0.015627,0.000000,0.013485,0.032915,0.011648,0.009587,0.000000,0.015199,0.024419,0.017094,0.029608,0.000000,0.000000,0.000000,0.000000,0.016093,0.006989,0.018970,0.048457,0.000000,0.023870,0.022422,0.00000,0.000000,0.000000,0.000000,0.042701,0.000000,0.000000,0.000000,0.000000,0.008336,0.000000,0.000000,0.000000
2,0.010186,0.020544,1.000000,0.013186,0.000000,0.000000,0.000000,0.011167,0.000000,0.011282,0.013186,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025325,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019342,0.000000,0.021092,0.000000,0.015559,0.000000,0.017623,0.009357,0.008757,0.013782,0.000000,0.000000,0.000000,0.011778,0.014621,...,0.000000,0.000000,0.000000,0.000000,0.041266,0.018781,0.000000,0.000000,0.000000,0.013999,0.000000,0.000000,0.000000,0.009783,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016798,0.045599,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.010879,0.000000,0.013186,1.000000,0.026053,0.019316,0.031439,0.023855,0.000000,0.000000,0.042254,0.018534,0.000000,0.013525,0.000000,0.021315,0.042862,0.000000,0.040574,0.025447,0.019445,0.014287,0.019445,0.042460,0.011694,0.020659,0.038171,0.033793,0.000000,0.016618,0.046327,0.009412,0.019989,0.037413,0.014720,0.033493,0.000000,0.025898,0.012580,0.015617,...,0.021315,0.000000,0.000000,0.000000,0.044076,0.020060,0.037069,0.000000,0.014085,0.000000,0.000000,0.000000,0.000000,0.010449,0.010972,0.019004,0.000000,0.000000,0.000000,0.000000,0.020659,0.017942,0.024352,0.074645,0.000000,0.000000,0.000000,0.00000,0.035783,0.000000,0.020659,0.000000,0.042630,0.014720,0.023736,0.019004,0.010701,0.032915,0.068519,0.013027
4,0.000000,0.010148,0.000000,0.026053,1.000000,0.017865,0.014539,0.000000,0.073432,0.000000,0.000000,0.000000,0.000000,0.000000,0.011906,0.000000,0.092498,0.000000,0.012509,0.011768,0.000000,0.000000,0.017984,0.019635,0.010815,0.000000,0.011768,0.031255,0.015370,0.000000,0.000000,0.000000,0.009244,0.008651,0.000000,0.000000,0.000000,0.000000,0.011635,0.007222,...,0.000000,0.007901,0.000000,0.000000,0.000000,0.018554,0.051427,0.000000,0.000000,0.000000,0.011382,0.000000,0.072181,0.000000,0.000000,0.000000,0.000000,0.000000,0.022887,0.000000,0.038215,0.000000,0.033785,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.019714,0.000000,0.000000,0.017576,0.000000,0.000000,0.000000,0.024096
5,0.000000,0.030094,0.000000,0.019316,0.017865,1.000000,0.00

In [161]:
df_cosine_sim[560:570]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9503,9504,9505,9506,9507,9508,9509,9510,9511,9512,9513,9514,9515,9516,9517,9518,9519,9520,9521,9522,9523,9524,9525,9526,9527,9528,9529,9530,9531,9532,9533,9534,9535,9536,9537,9538,9539,9540,9541,9542
560,0.000000,0.024932,0.000000,0.016003,0.000000,0.021946,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032466,0.000000,0.000000,0.000000,0.000000,0.011047,0.048242,0.000000,0.046945,0.028913,0.000000,0.037763,0.000000,0.026318,0.010694,0.011356,0.000000,0.000000,0.038054,0.000000,0.000000,0.000000,0.008872,...,0.000000,0.077648,0.017264,0.000000,0.025039,0.022792,0.000000,0.000000,0.016003,0.050965,0.000000,0.000000,0.000000,0.023744,0.000000,0.043183,0.0,0.000000,0.000000,0.000000,0.000000,0.020386,0.000000,0.042405,0.000000,0.000000,0.032703,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029601
561,0.011934,0.012036,0.028931,0.030901,0.000000,0.074162,0.000000,0.039253,0.000000,0.000000,0.015451,0.000000,0.016949,0.000000,0.014121,0.046765,0.047019,0.062692,0.014836,0.000000,0.053327,0.015673,0.042662,0.011644,0.038484,0.045326,0.013958,0.049428,0.000000,0.036460,0.012705,0.010325,0.010964,0.010260,0.016148,0.012247,0.052076,0.056819,0.027600,0.025697,...,0.000000,0.018742,0.033338,0.000000,0.024175,0.088024,0.060996,0.018990,0.000000,0.000000,0.054000,0.000000,0.000000,0.057312,0.012036,0.020847,0.0,0.052076,0.027146,0.046765,0.022663,0.000000,0.013357,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.050847,0.000000,0.045099,0.000000,0.064592,0.000000,0.020847,0.023477,0.000000,0.000000,0.000000
562,0.000000,0.000000,0.000000,0.038007,0.017576,0.013031,0.021209,0.048280,0.013391,0.000000,0.038007,0.025008,0.020847,0.000000,0.000000,0.028760,0.038554,0.000000,0.018248,0.017168,0.000000,0.019277,0.013118,0.028645,0.015778,0.000000,0.017168,0.045596,0.022422,0.022422,0.015627,0.000000,0.013485,0.037860,0.019861,0.030127,0.000000,0.034943,0.016974,0.031607,...,0.000000,0.000000,0.000000,0.000000,0.029735,0.000000,0.000000,0.000000,0.019004,0.000000,0.000000,0.000000,0.000000,0.042295,0.014804,0.025641,0.0,0.000000,0.000000,0.000000,0.027875,0.000000,0.000000,0.000000,0.053376,0.000000,0.038837,0.0,0.000000,0.000000,0.027875,0.000000,0.028760,0.019861,0.000000,0.000000,0.014438,0.000000,0.000000,0.000000
563,0.000000,0.011648,0.000000,0.014952,0.013829,0.041011,0.000000,0.012662,0.000000,0.000000,0.029904,0.000000,0.000000,0.028715,0.000000,0.022628,0.045502,0.000000,0.000000,0.000000,0.000000,0.015167,0.010321,0.045075,0.037242,0.021932,0.013507,0.023917,0.088209,0.017642,0.012295,0.029975,0.010610,0.019859,0.000000,0.011852,0.000000,0.027493,0.026709,0.033158,...,0.000000,0.045344,0.048393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.055463,0.034943,0.020174,0.0,0.000000,0.000000,0.022628,0.021932,0.000000,0.000000,0.052829,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.021932,0.029096,0.000000,0.015627,0.025198,0.000000,0.000000,0.000000,0.000000,0.000000
564,0.035284,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022840,0.000000,0.025055,0.021932,0.000000,0.034565,0.023168,0.000000,0.000000,0.000000,0.015766,0.000000,0.015766,0.017213,0.056888,0.033501,0.000000,0.018267,0.000000,0.000000,0.000000,0.000000,0.016207,0.060669,0.000000,0.000000,0.000000,0.000000,0.000000,0.063311,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030056,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016944,0.017792,0.000000,0.0,0.000000,0.000000,0.034565,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.058026,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030817,0.017353,0.000000,0.000000,0.000000
565,0.009213,0.009292,0.022334,0.011928,0.000000,0.008179,0.013312,0.060606,0.033618,0.061228,0

In [162]:
# replace the index and column names, which were serielly numbered, with the movie Ids of movielens
df_cosine_sim = df_cosine_sim.set_axis(df_features.index, axis=1)
df_cosine_sim = df_cosine_sim.set_axis(df_features.index, axis=0)

In [163]:
df_cosine_sim.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,173255,173291,173307,173317,173351,173355,173619,173751,173873,173925,173941,173963,174045,174053,174055,174141,174403,174479,174551,174681,174727,174737,174815,175197,175293,175387,175397,175401,175431,175435,175475,175569,175577,175585,175693,175705,175707,175743,175781,176051
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.033900,0.010186,0.010879,0.000000,0.000000,0.012142,0.046066,0.000000,0.000000,0.021758,0.014316,0.023869,0.000000,0.000000,0.016464,0.011036,0.044143,0.010447,0.039312,0.022530,0.000000,0.015020,0.024598,0.018065,0.015958,0.019656,0.000000,0.012836,0.025673,0.008946,0.000000,0.030880,0.000000,0.022740,0.017247,0.000000,0.000000,0.000000,0.024126,...,0.032929,0.000000,0.011737,0.000000,0.051068,0.000000,0.028633,0.000000,0.000000,0.000000,0.000000,0.023869,0.015070,0.024213,0.000000,0.014679,0.000000,0.000000,0.000000,0.000000,0.000000,0.006930,0.028215,0.000000,0.030557,0.023669,0.022233,0.09167,0.110558,0.011934,0.015958,0.000000,0.016464,0.000000,0.018334,0.014679,0.008266,0.025425,0.052926,0.000000
2,0.033900,1.000000,0.020544,0.000000,0.010148,0.030094,0.000000,0.009292,0.100504,0.009387,0.021944,0.014438,0.000000,0.010536,0.000000,0.000000,0.011130,0.044519,0.000000,0.029735,0.007574,0.000000,0.022721,0.016538,0.009109,0.016093,0.009912,0.008775,0.051782,0.000000,0.009022,0.021995,0.015571,0.000000,0.011467,0.008697,0.000000,0.000000,0.009800,0.012166,...,0.016605,0.019964,0.000000,0.014804,0.017168,0.015627,0.000000,0.013485,0.032915,0.011648,0.009587,0.000000,0.015199,0.024419,0.017094,0.029608,0.000000,0.000000,0.000000,0.000000,0.016093,0.006989,0.018970,0.048457,0.000000,0.023870,0.022422,0.00000,0.000000,0.000000,0.000000,0.042701,0.000000,0.000000,0.000000,0.000000,0.008336,0.000000,0.000000,0.000000
3,0.010186,0.020544,1.000000,0.013186,0.000000,0.000000,0.000000,0.011167,0.000000,0.011282,0.013186,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025325,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019342,0.000000,0.021092,0.000000,0.015559,0.000000,0.017623,0.009357,0.008757,0.013782,0.000000,0.000000,0.000000,0.011778,0.014621,...,0.000000,0.000000,0.000000,0.000000,0.041266,0.018781,0.000000,0.000000,0.000000,0.013999,0.000000,0.000000,0.000000,0.009783,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016798,0.045599,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.010879,0.000000,0.013186,1.000000,0.026053,0.019316,0.031439,0.023855,0.000000,0.000000,0.042254,0.018534,0.000000,0.013525,0.000000,0.021315,0.042862,0.000000,0.040574,0.025447,0.019445,0.014287,0.019445,0.042460,0.011694,0.020659,0.038171,0.033793,0.000000,0.016618,0.046327,0.009412,0.019989,0.037413,0.014720,0.033493,0.000000,0.025898,0.012580,0.015617,...,0.021315,0.000000,0.000000,0.000000,0.044076,0.020060,0.037069,0.000000,0.014085,0.000000,0.000000,0.000000,0.000000,0.010449,0.010972,0.019004,0.000000,0.000000,0.000000,0.000000,0.020659,0.017942,0.024352,0.074645,0.000000,0.000000,0.000000,0.00000,0.035783,0.000000,0.020659,0.000000,0.042630,0.014720,0.023736,0.019004,0.010701,0.032915,0.068519,0.013027
5,0.000000,0.010148,0.000000,0.026053,1.000000,0.017865,0.014539,0.000000,0.073432,0.000000,0.000000,0.000000,0.000000,0.000000,0.011906,0.000000,0.092498,0.000000,0.012509,0.011768,0.000000,0.000000,0.017984,0.019635,0.010815,0.000000,0.011768,0.031255,0.015370,0.000000,0.000000,0.000000,0.009244,0.008651,0.000000,0.000000,0.000000,0.000000,0.011635,0.007222,...,0.000000,0.007901,0.000000,0.000000,0.000000,0.018554,0.051427,0.000000,0.000000,0.000000,0.011382,0.000000,0.072181,0.000000,0.000000,0.000000,0.000000,0.000000,0.022887,0.000000,0.038215,0.000000,0.033785,0.000000,0.000000,0.000000,0.000000

In [164]:
# sample query a cosine similarity between two movies
df_cosine_sim.loc[1,175781]

0.052925612402496325

## In order to generate a .csv of the cosine similarity dataframe, unhash the code below
-> also change the directory path according to your needs

In [165]:
# export new data to csv. file
#df_cosine_sim.to_csv('../data/cos_sim_matrix.csv',index=True)

### Save data as .NPZ (compressed)
The generated .csv file is 1,2 GB, so far too big. The better way is to store it as a numpy array, because it is only needed in a .py script. Unhash the code below, if needed

In [166]:
# save numpy array as npz file
#from numpy import asarray
#from numpy import savez_compressed

# save to npy file
#savez_compressed('../data/cosine_sim.npz', cosine_sim)